# Импорт библиотек

In [3]:
import random
import nltk
from pptx import Presentation
from pptx.exc import PackageNotFoundError
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords, wordnet

# Убедитесь, что NLTK и WordNet установлены и загружены необходимые ресурсы
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Анализ презентации и извлечение из нее слов

In [4]:
def extract_text_from_presentation(file_path):
    '''
    Извлекает текст из презентации.
    '''
    text_runs = []
    try:
        prs = Presentation(file_path)
        for slide in prs.slides:
            for shape in slide.shapes:
                if not shape.has_text_frame:
                    continue
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        text_runs.append(run.text)
    except PackageNotFoundError:
        print(f"Файл {file_path} не найден.")
        return None
    except Exception as e:
        print(f"Произошла ошибка при чтении файла: {e}")
        return None
    return ' '.join(text_runs)


# Поиск ключевых слов для создания вопросов квиза

In [5]:
def select_key_word(sentence):
    '''
    Выбирает ключевое слово из предложения.
    '''
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    stop_words = set(stopwords.words('english'))
    significant_words = [word for word, tag in tagged_words if word.lower() not in stop_words and tag in ['NN', 'VB', 'JJ']]
    return random.choice(significant_words) if significant_words else random.choice(words)


# Создание вопросов квиза

In [6]:
def create_quiz_questions(text, num_questions=5):
    '''
    Генерирует вопросы для викторины.
    '''
    sentences = sent_tokenize(text)
    questions = []
    for i in range(num_questions):
        sentence = random.choice(sentences)
        answer = select_key_word(sentence)
        if answer not in sentence:
            continue
        question = sentence.replace(answer, '_____')
        questions.append({'question': question, 'answer': answer})
    return questions

# Разбиение слов по семантике

In [7]:
def get_semantically_similar_words(word, num_words=3):
    '''
    Возвращает семантически близкие слова.
    '''
    synonyms = set()
    antonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
            if lemma.antonyms():
                antonyms.add(lemma.antonyms()[0].name())
    similar_words = list(synonyms.union(antonyms))
    if word in similar_words:
        similar_words.remove(word)
    return random.sample(similar_words, min(len(similar_words), num_words))

# Создание ответов для квиза

In [8]:
def create_answers(correct_answer, context_sentences, num_incorrect_answers=3):
    '''
    Создает список ответов для викторины.
    '''
    incorrect_answers = get_semantically_similar_words(correct_answer, num_incorrect_answers)
    while len(incorrect_answers) < num_incorrect_answers:
        sample_sentence = random.choice(context_sentences)
        if correct_answer not in sample_sentence:
            incorrect_answers.append(sample_sentence)
    options = incorrect_answers + [correct_answer]
    random.shuffle(options)
    return options

# Создание полного квиза

In [9]:
def generate_full_quiz(questions):
    '''
    Генерирует полный квиз.
    '''
    quiz = []
    for q in questions:
        context_sentences = sent_tokenize(q['context'])
        correct_answer = q['answer']
        question = q['question']
        options = create_answers(correct_answer, context_sentences)
        quiz.append({'question': question, 'options': options, 'correct_answer': correct_answer})
    return quiz

# Оценка ответов пользователей

In [10]:
def evaluate_quiz(quiz, user_answers):
    '''
    Оценивает ответы пользователя.
    '''
    score = 0
    for question, user_answer in zip(quiz, user_answers):
        if user_answer == question['correct_answer']:
            score += 1
    return score

# Основной блок кода

In [11]:
if __name__ == "__main__":
    presentation_file_path = 'path_to_presentation.pptx'
    extracted_text = extract_text_from_presentation(presentation_file_path)
    if extracted_text:
        quiz_questions = create_quiz_questions(extracted_text)
        full_quiz = generate_full_quiz(quiz_questions)
        user_answers = ['ответ1', 'ответ2', 'ответ3']  # Заполните своими данными
        final_score = evaluate_quiz(full_quiz, user_answers)
        print(f"Ваш итоговый счёт: {final_score}")

Файл path_to_presentation.pptx не найден.


Последние улучшения кода:

* Устранено дублирование функций.
* Оптимизирован выбор неправильных ответов.
* Улучшена обработка исключений.
* Изменена логика оценки, чтобы избежать отрицательных баллов.
* Добавлены проверки наличия ключевого слова в предложении.
* Улучшены комментарии и документация.
* Предполагается, что вы проведете тестирование каждой функции.

# Документация к коду телеграмм-бота для викторин

## Обзор
Этот код предназначен для создания и оценки викторин на основе текста, извлеченного из презентаций. Он включает функции для извлечения текста, генерации вопросов и ответов, создания полного квиза и оценки ответов пользователя.

## Функции

### `extract_text_from_presentation(file_path)`
Извлекает текст из презентации PowerPoint. Возвращает строку с объединенным текстом всех слайдов или `None` в случае ошибки.

#### Параметры:
- `file_path`: Путь к файлу презентации.

#### Возвращаемое значение:
- Строка с текстом презентации или `None`.

### `select_key_word(sentence)`
Выбирает ключевое слово из предложения для использования в вопросе викторины.

#### Параметры:
- `sentence`: Предложение для выбора ключевого слова.

#### Возвращаемое значение:
- Строка с ключевым словом.

### `create_quiz_questions(text, num_questions=5)`
Генерирует список вопросов для викторины на основе извлеченного текста.

#### Параметры:
- `text`: Текст для генерации вопросов.
- `num_questions`: Количество генерируемых вопросов (по умолчанию 5).

#### Возвращаемое значение:
- Список словарей с вопросами и ответами.

### `get_semantically_similar_words(word, num_words=3)`
Находит семантически близкие слова к заданному слову.

#### Параметры:
- `word`: Слово для поиска семантически близких слов.
- `num_words`: Количество возвращаемых слов (по умолчанию 3).

#### Возвращаемое значение:
- Список семантически близких слов.

### `create_answers(correct_answer, context_sentences, num_incorrect_answers=3)`
Создает список ответов для викторины, включая правильный и неправильные ответы.

#### Параметры:
- `correct_answer`: Правильный ответ.
- `context_sentences`: Список предложений для выбора неправильных ответов.
- `num_incorrect_answers`: Количество неправильных ответов (по умолчанию 3).

#### Возвращаемое значение:
- Список с вариантами ответов.

### `generate_full_quiz(questions)`
Генерирует полный квиз, включая вопросы и варианты ответов.

#### Параметры:
- `questions`: Список вопросов для викторины.

#### Возвращаемое значение:
- Список словарей с полными вопросами викторины.

### `evaluate_quiz(quiz, user_answers)`
Оценивает ответы пользователя и возвращает итоговый счет.

#### Параметры:
- `quiz`: Список вопросов викторины.
- `user_answers`: Список ответов пользователя.

#### Возвращаемое значение:
- Целое число, представляющее итоговый счет пользователя.

## Использование
Чтобы использовать этот код, необходимо запустить его как основной скрипт, указав путь к файлу презентации. После извлечения текста код автоматически сгенерирует вопросы, создаст квиз и оценит ответы пользователя.

## Тестирование
Перед использованием рекомендуется провести тестирование каждой функции отдельно и вместе, чтобы убедиться в их корректной работе.
